In [34]:
import gym
from matplotlib import pyplot as plt
%matplotlib inline

#创建环境
env = gym.make('CartPole-v0')
env.reset()


#打印游戏
def show():
    env.render(mode='rgb_array')


show()

Exception ignored in: <function Viewer.__del__ at 0x000002667BA29620>
Traceback (most recent call last):
  File "D:\python\Python311\Lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "D:\python\Python311\Lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "D:\python\Python311\Lib\site-packages\pyglet\window\win32\__init__.py", line 313, in close
    super(Win32Window, self).close()
  File "D:\python\Python311\Lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "D:\python\Python311\Lib\_weakrefset.py", line 113, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000026606EAD1C0; to 'Win32Window' at 0x0000026608DC9B10>


In [35]:
#测试游戏环境
def test_env():
    state = env.reset()
    print('这个游戏的状态用4个数字表示,我也不知道这4个数字分别是什么意思,反正这4个数字就能描述游戏全部的状态')
    print('state=', state)
    #state= [ 0.03490619  0.04873464  0.04908862 -0.00375859]

    print('这个游戏一共有2个动作,不是0就是1')
    print('env.action_space=', env.action_space)
    #env.action_space= Discrete(2)

    print('随机一个动作')
    action = env.action_space.sample()
    print('action=', action)
    #action= 1

    print('执行一个动作,得到下一个状态,奖励,是否结束')
    state, reward, over, _ = env.step(action)

    print('state=', state)
    #state= [ 0.02018229 -0.16441101  0.01547085  0.2661691 ]

    print('reward=', reward)
    #reward= 1.0

    print('over=', over)
    #over= False


test_env()

这个游戏的状态用4个数字表示,我也不知道这4个数字分别是什么意思,反正这4个数字就能描述游戏全部的状态
state= [ 0.03645723 -0.00906221  0.03182828 -0.03750419]
这个游戏一共有2个动作,不是0就是1
env.action_space= Discrete(2)
随机一个动作
action= 0
执行一个动作,得到下一个状态,奖励,是否结束
state= [ 0.03627598 -0.20462578  0.03107819  0.26504853]
reward= 1.0
over= False


In [36]:
import torch

#计算动作的模型,也是真正要用的模型
model = torch.nn.Sequential(
    torch.nn.Linear(4, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 2),
)

model

Sequential(
  (0): Linear(in_features=4, out_features=16, bias=True)
  (1): ReLU()
  (2): Linear(in_features=16, out_features=2, bias=True)
)

In [37]:
import random


#得到一个动作
def get_action(state):
    if random.random() < 0.01:
        return random.choice([0, 1])

    #走神经网络,得到一个动作
    state = torch.FloatTensor(state).reshape(1, 4)

    return model(state).argmax().item()


get_action([0.0013847, -0.01194451, 0.04260966, 0.00688801])

0

In [38]:
#样本池
datas = []


#向样本池中添加N条数据,删除M条最古老的数据
def update_data():
    old_count = len(datas)

    #玩到新增了N个数据为止
    while len(datas) - old_count < 200:
        #初始化游戏
        state = env.reset()

        #玩到游戏结束为止
        over = False
        while not over:
            #根据当前状态得到一个动作
            action = get_action(state)

            #执行动作,得到反馈
            next_state, reward, over, _ = env.step(action)

            #记录数据样本
            datas.append((state, action, reward, next_state, over))

            #更新游戏状态,开始下一个动作
            state = next_state

    update_count = len(datas) - old_count
    drop_count = max(len(datas) - 10000, 0)

    #数据上限,超出时从最古老的开始删除
    while len(datas) > 10000:
        datas.pop(0)

    return update_count, drop_count


update_data(), len(datas)

((203, 0), 203)

In [39]:
#获取一批数据样本
def get_sample():
    #从样本池中采样
    samples = random.sample(datas, 64)

    #[b, 4]
    state = torch.FloatTensor([i[0] for i in samples])
    #[b]
    action = torch.LongTensor([i[1] for i in samples])
    #[b]
    reward = torch.FloatTensor([i[2] for i in samples])
    #[b, 4]
    next_state = torch.FloatTensor([i[3] for i in samples])
    #[b]
    over = torch.LongTensor([i[4] for i in samples])

    return state, action, reward, next_state, over


state, action, reward, next_state, over = get_sample()

# state[:5], action, reward, next_state[:5], over

In [40]:
def get_value(state, action):
    #使用状态计算出动作的logits
    #[b, 4] -> [b, 2]
    value = model(state)

    #根据实际使用的action取出每一个值
    #这个值就是模型评估的在该状态下,执行动作的分数
    #在执行动作前,显然并不知道会得到的反馈和next_state
    #所以这里不能也不需要考虑next_state和reward
    #[b, 2] -> [b]
    value = value[range(64), action]

    return value


get_value(state, action)

tensor([-0.0611,  0.0598,  0.0705, -0.0352,  0.0931, -0.0755, -0.1808,  0.1001,
        -0.0226,  0.0819,  0.0828, -0.0688, -0.1497,  0.0818, -0.0485, -0.0841,
         0.0994, -0.0369, -0.0736,  0.0889, -0.1180,  0.0871,  0.0607,  0.0747,
         0.0045, -0.1385, -0.1587,  0.0126, -0.1026,  0.0888,  0.0531, -0.1079,
        -0.0960,  0.0347,  0.0905, -0.1280,  0.0948,  0.0189,  0.0405,  0.0786,
        -0.1283,  0.0735, -0.1047, -0.0329, -0.0849, -0.1369, -0.1626,  0.0135,
         0.0689, -0.1424,  0.0856,  0.0237, -0.1179,  0.0742, -0.0809, -0.1855,
         0.0656, -0.0314, -0.0563,  0.0945, -0.0850,  0.0781,  0.0621,  0.0407],
       grad_fn=<IndexBackward0>)

In [41]:
def get_target(reward, next_state, over):
    #上面已经把模型认为的状态下执行动作的分数给评估出来了
    #下面使用next_state和reward计算真实的分数
    #针对一个状态,它到底应该多少分,可以使用以往模型积累的经验评估
    #这也是没办法的办法,因为显然没有精确解,这里使用延迟更新的next_model评估

    #使用next_state计算下一个状态的分数
    #[b, 4] -> [b, 2]
    with torch.no_grad():
        target = model(next_state)

    #取所有动作中分数最大的
    #[b, 2] -> [b]
    target = target.max(dim=1)[0]

    #如果next_state已经游戏结束,则next_state的分数是0
    #因为如果下一步已经游戏结束,显然不需要再继续玩下去,也就不需要考虑next_state了.
    #[b]
    for i in range(64):
        if over[i]:
            target[i] = 0

    #下一个状态的分数乘以一个系数,相当于权重
    #[b] * [b] -> [b]
    target *= 0.98

    #加上reward就是最终的分数
    #[b] + [b] -> [b]
    target += reward

    return target


get_target(reward, next_state, over)

tensor([0.9037, 1.0551, 1.0642, 0.9288, 1.0448, 0.8897, 1.0000, 1.0960, 0.9419,
        1.0761, 1.0763, 0.8994, 0.8190, 1.0770, 0.9208, 0.8818, 1.0959, 0.9260,
        0.8943, 1.0848, 0.8551, 1.0826, 1.0552, 1.0695, 0.9499, 1.0000, 0.8145,
        0.9774, 0.8694, 1.0832, 1.0477, 0.8642, 0.8742, 0.9879, 1.0841, 0.8407,
        1.0908, 0.9639, 0.9905, 1.0399, 1.0000, 1.0657, 1.0000, 0.9335, 0.8844,
        1.0000, 1.0000, 0.9645, 1.0629, 0.8287, 1.0814, 0.9747, 0.8486, 1.0689,
        0.8863, 1.0000, 1.0132, 0.9326, 0.9135, 1.0889, 0.8816, 1.0742, 1.0185,
        0.9885])

In [42]:
from IPython import display


def test(play):
    #初始化游戏
    state = env.reset()

    #记录反馈值的和,这个值越大越好
    reward_sum = 0

    #玩到游戏结束为止
    over = False
    while not over:
        #根据当前状态得到一个动作
        action = get_action(state)

        #执行动作,得到反馈
        state, reward, over, _ = env.step(action)
        reward_sum += reward

        #打印动画
        if play:
            display.clear_output(wait=True)
            show()

    return reward_sum


test(play=False)

9.0

In [44]:
def train():
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
    loss_fn = torch.nn.MSELoss()

    #训练N次
    for epoch in range(1):
        #更新N条数据
        update_count, drop_count = update_data()

        #每次更新过数据后,学习N次
        for i in range(1):
            #采样一批数据
            state, action, reward, next_state, over = get_sample()

            #计算一批样本的value和target
            value = get_value(state, action)
            target = get_target(reward, next_state, over)

            #更新参数
            loss = loss_fn(value, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if epoch % 50 == 0:
            test_result = sum([test(play=False) for _ in range(20)]) / 20
            print(epoch, len(datas), update_count, drop_count, test_result)


train()

0 408 205 0 9.2


In [14]:
test(play=True)

87.0

In [43]:
[*model.parameters()][0]

Parameter containing:
tensor([[-0.0112,  0.1397, -0.2651,  0.1428],
        [-0.2196, -0.3746, -0.2789, -0.0629],
        [-0.1487, -0.3741, -0.2815,  0.2363],
        [ 0.2737,  0.4999,  0.1023,  0.2207],
        [-0.0982,  0.3085, -0.0156, -0.2932],
        [-0.4242,  0.3416,  0.2880, -0.3576],
        [ 0.0163, -0.1218, -0.0212,  0.3527],
        [-0.0578, -0.3019, -0.2483,  0.3652],
        [-0.0698, -0.2277, -0.1791, -0.2455],
        [-0.0932, -0.1505,  0.0902,  0.2408],
        [-0.0397,  0.2067, -0.0609,  0.3886],
        [-0.1679, -0.4822,  0.2225, -0.0483],
        [ 0.0959, -0.1113,  0.0410,  0.1039],
        [ 0.4980, -0.0071,  0.3389,  0.4030],
        [ 0.0539, -0.3029,  0.4025, -0.1725],
        [-0.2776,  0.1995, -0.3249,  0.4110]], requires_grad=True)

In [45]:
[*model.parameters()][0]

Parameter containing:
tensor([[-0.0092,  0.1417, -0.2671,  0.1408],
        [-0.2176, -0.3726, -0.2809, -0.0649],
        [-0.1467, -0.3721, -0.2835,  0.2343],
        [ 0.2737,  0.4999,  0.1023,  0.2207],
        [-0.0982,  0.3085, -0.0156, -0.2932],
        [-0.4242,  0.3416,  0.2880, -0.3576],
        [ 0.0143, -0.1238, -0.0192,  0.3547],
        [-0.0558, -0.2999, -0.2503,  0.3632],
        [-0.0678, -0.2257, -0.1811, -0.2475],
        [-0.0952, -0.1525,  0.0922,  0.2428],
        [-0.0377,  0.2087, -0.0629,  0.3866],
        [-0.1659, -0.4802,  0.2205, -0.0503],
        [ 0.0939, -0.1133,  0.0430,  0.1059],
        [ 0.4960, -0.0091,  0.3409,  0.4050],
        [ 0.0539, -0.3029,  0.4025, -0.1725],
        [-0.2796,  0.1975, -0.3229,  0.4130]], requires_grad=True)